# Data As Code Example

Alice wants to produce a CSV file which describes the number of available beds
in each hospital, each week, in Washington state, then send it to Bob.
She finds a CSV on a government website that has these data, along with other
data that she doesn't want. She needs to manipulate the file to include only the
columns and rows that she is interested in.

https://healthdata.gov/sites/default/files/reported_hospital_capacity_admissions_facility-level_weekly_average_timeseries_20201207.csv

In a common workflow, Alice would download the file with a browser, use some
tools to filter rows and columns, then output to a new CSV. Then she would send
the file to Bob in an email. Many users would do this in a visual tool like
Excel, but Alice really likes `pandas`, so she'll use that to manipulate the
file. She could easily do this from a python console, but she writes a script
like this:


In [24]:
from pathlib import Path

import pandas as pd

dlp = Path(Path.home(), 'AliceAnalysis', 'beds_20201207.csv')
dlp_df = pd.read_csv(dlp)
wanted_column = ['collection_week', 'hospital_name', 'all_adult_hospital_beds_7_day_avg']
new_df = dlp_df[dlp_df['state'] == 'WA'][wanted_column]
new_df.to_csv(
    Path(Path.home(), 'AliceAnalysis', 'out.csv'),
    index=False
)

When Alice downloaded the file, she shortened the name to `beds_20201207.csv`
because the original was too long; it wasn't easy to reference, and she didn't
like the look of it on her file system. She kept the full URL for reference.
Alice used `pandas` to load the file that she downloaded manually, performed her
manipulations, then wrote the result to another `csv`. Now she'll attach
`out.csv` in an email, but first she renames it to `wa_bed_average_20210201`.
Then she sends it to Bob.

This process works *ok* if Bob doesn't care where the data comes from. In most
cases, Bob really does care about how the data were produced, and he insists on
an explanation of sources and manipulations. So Alice will also need to include
a lengthy description in her email of where she got the data, what she did to
it, and so on. She could write all this in english in her email, but she decides
that her script has plenty of description, so she attaches that instead. So now
she has an email with:

 1. a URL to download the file (with a different, longer file name)
 2. a python script with esoteric file paths, including her renamed `csv`
 3. an attached `csv` with the result

This helps Bob understand the data, a little, but the script is only so useful;
he can't run the script himself without updating file paths and names. If Bob
is just interested in seeing the code, then that's ok, but the code is not
reusable in its current form. Bob can't easily verify that the file reference
in the URL is the same one that was used to produce the attached file. Bob's
also not certain if Alice actually attached the correct output file. There were
numerous manual steps involved in the process, which raises questions about
whether the attached code actually lines up with the ultimate product of the
process.

This is a common problem, which can be solved by treating **data as code**;
instead of incorporating numerous manual steps in a workflow to create a data
product, Alice can instead script the entire process, and generate an exhaustive
recipe - complete with metadata for every step - with verifiable checksums for
all inputs and outputs in the process.


In [1]:
from pathlib import Path

import pandas as pd
from data_as_code import Recipe, step

with Recipe() as r:
    step.SourceHTTP(
        r, ('https://healthdata.gov/sites/default/files/'
        'reported_hospital_capacity_admissions_facility-level_weekly_average_timeseries_20201207.csv'
         ), name='admits'
    )

    class Subset(step.Step):
        admits = step.Input('admits')

        def instructions(self):
            df = pd.read_csv(self.admits.path)
            columns = ['collection_week', 'hospital_name', 'all_adult_hospital_beds_7_day_avg']
            df = df[df['state'] == 'WA'][columns]
            p = Path('washington_adult_7_day_bed_average.csv')
            df.to_csv(p, index=False)
            return p

    Subset(r)
    r.add_product('washington_adult_7_day_bed_average.csv')